In [1]:
from PIL import Image
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

import sys, os
depth_anything_path = "/home/harry7557558/GitHub/external/Depth-Anything/"
sys.path += [depth_anything_path]
current_directory = os.getcwd()
os.chdir(depth_anything_path)

from depth_anything.dpt import DepthAnything

encoder = 'vits' # can also be 'vitb' or 'vitl'
model = DepthAnything.from_pretrained('LiheYoung/depth_anything_{:}14'.format(encoder))
model = model.to(device)

os.chdir(current_directory)

# print(model)

/home/harry7557558/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
xFormers not available
xFormers not available


In [4]:
image_filename = "/media/harry7557558/OS/Archive/DCIM/20230415_093519.jpg"
# image_filename = "/home/harry7557558/nerd-face_1f913.png"
img = Image.open(image_filename)

from PIL import ExifTags
for orientation in ExifTags.TAGS.keys():
    if ExifTags.TAGS[orientation]=='Orientation':
        break
if hasattr(img, '_getexif'):
    exif = img._getexif()
    if exif is not None and orientation in exif:
        print("exif orientation:", exif[orientation])
        if exif[orientation] == 3:
            img=img.rotate(180, expand=True)
        elif exif[orientation] == 6:
            img=img.rotate(270, expand=True)
        elif exif[orientation] == 8:
            img=img.rotate(90, expand=True)

x = img.convert("RGB")
scale = 1024/(x.width*x.height)**0.5
w = int(x.width*scale/14+0.5)*14
h = int(x.height*scale/14+0.5)*14
x = x.resize((w, h), Image.LANCZOS)

print("Image size:", x.size)

x = np.array(x, dtype=np.float32) / 255.0
x = np.transpose(x, (2, 0, 1))

from time import perf_counter
with torch.no_grad():
    time0 = perf_counter()
    
    mu = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((3, 1, 1))
    sigma = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((3, 1, 1))
    xc = torch.tensor((x-mu)/sigma, device=device).unsqueeze(0)
    y = model(xc)[0].cpu().numpy()

    time1 = perf_counter()
    print(1000*(time1-time0), 'ms')


exif orientation: 1
Image size: (1176, 882)
276.6394109930843 ms


In [8]:
from plyfile import PlyData, PlyElement

def write_ply(rgb_array, depth_array, file_path):
    h, w = depth_array.shape
    num_points = w * h

    # Flatten arrays
    depth_flat = depth_array.flatten()
    rgb_flat = rgb_array.reshape((3, -1)).T

    # Create vertex data
    vertex = np.zeros(num_points, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), 
                                         ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])
    vertex['x'] = np.tile(np.arange(w), h) / (w*h)**0.5
    vertex['y'] = w - np.repeat(np.arange(h), w) / (w*h)**0.5
    # vertex['z'] = depth_flat / (2.5*np.std(depth_flat))
    vertex['z'] = depth_flat / (3.0*np.mean(depth_flat))
    rgb = (np.clip(rgb_flat,0,1)*255+0.5).astype(np.uint8)
    vertex['red'] = rgb[:, 0]
    vertex['green'] = rgb[:, 1]
    vertex['blue'] = rgb[:, 2]

    faces = []
    depth = vertex['z']
    print(depth.shape)
    for y in range(h - 1):
        for x in range(w - 1):
            i00 = y * w + x
            i10 = i00 + 1
            i01 = (y + 1) * w + x
            i11 = i01 + 1
            gx = depth[i01]+depth[i11]-depth[i00]-depth[i10]
            gy = depth[i10]+depth[i11]-depth[i00]-depth[i01]
            if np.hypot(gx, gy) < 20 / (w*h)**0.5:
                faces.append(([i00, i10, i11, i01], 0))

    faces = np.array(faces,
                     dtype=[('vertex_indices', 'i4', (4,)), ('i', 'u1')])

    # Create PlyElements
    vertex_element = PlyElement.describe(vertex, 'vertex')
    face_element = PlyElement.describe(faces, 'face')

    # Write to PLY file
    ply_data = PlyData([vertex_element, face_element])
    ply_data.write(file_path)


from scipy.ndimage import zoom

scale = 384/(x.shape[1]*x.shape[2])**0.5
x = zoom(x, (1, scale, scale))
y = zoom(y, (scale, scale))
write_ply(x, y, "depth_mesh.ply")


(147519,)
